# Maze Example

The creation of a maze implemented by applying multiple design patterns.

The maze is composed by rooms, bounded by walls and doors that estabilish the communication between two adjacent rooms.

## Base Classes

In [2]:
from abc import ABC, abstractmethod
from enum import Enum

# Direction Enum
class Direction(Enum):
    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3

# MapSite Abstract Base Class
class MapSite(ABC):
    @abstractmethod
    def Enter(self):
        pass

# Wall Class
class Wall(MapSite):
    def Enter(self):
        print("You just ran into a wall.")

# Room Class
class Room(MapSite):
    def __init__(self, roomNo: int):
        self._roomNumber = roomNo
        self._sides = [None] * 4

    def GetSide(self, direction: Direction) -> MapSite:
        return self._sides[direction.value]

    def SetSide(self, direction: Direction, site: MapSite):
        self._sides[direction.value] = site

    def Enter(self):
        print(f"Entering room {self._roomNumber}")

# Door Class
class Door(MapSite):
    def __init__(self, room1: Room = None, room2: Room = None):
        self._room1 = room1
        self._room2 = room2
        self._isOpen = False

    def Enter(self):
        if self._isOpen:
            print("You pass through the door.")
        else:
            print("The door is closed.")

    def OtherSideFrom(self, room: Room) -> Room:
        if room == self._room1:
            return self._room2
        elif room == self._room2:
            return self._room1
        else:
            return None
        
# Maze Class
class Maze:
    def __init__(self):
        self._rooms = {}

    def AddRoom(self, room: Room):
        self._rooms[room._roomNumber] = room

    def RoomNo(self, room_number: int) -> Room:
        return self._rooms.get(room_number)

## Hard-Coded Implementation

In [ ]:
class MazeGame:
    @staticmethod
    def CreateMaze() -> Maze:
        aMaze = Maze()
        r1 = Room(1)
        r2 = Room(2)
        theDoor = Door(r1, r2)
        
        aMaze.AddRoom(r1)
        aMaze.AddRoom(r2)
        
        r1.SetSide(Direction.NORTH, Wall())
        r1.SetSide(Direction.EAST, theDoor)
        r1.SetSide(Direction.SOUTH, Wall())
        r1.SetSide(Direction.WEST, Wall())
        
        r2.SetSide(Direction.NORTH, Wall())
        r2.SetSide(Direction.EAST, Wall())
        r2.SetSide(Direction.SOUTH, Wall())
        r2.SetSide(Direction.WEST, theDoor)
        
        return aMaze
    
maze = MazeGame.CreateMaze()
room1 = maze.RoomNo(1)
room2 = maze.RoomNo(2)

print(f"Room 1 sides: {[type(room1.GetSide(direction)).__name__ for direction in Direction]}")
print(f"Room 2 sides: {[type(room2.GetSide(direction)).__name__ for direction in Direction]}")

## Abstract Factory

Implementation of the class `MazeFactory` that can create the components of a maze.

The implmentation of `CreateMaze` allows creating mazes with different components by taking `MazeFactory` as parameter.

The `MazeFactory` is just a collection of factory methods. This is the most common way to implement the `Abstract Factory` pattern. 

In [ ]:
class MazeFactory:
    def MakeMaze(self):
        return Maze()
    
    def MakeWall(self):
        return Wall()
    
    def MakeRoom(self, n: int):
        return Room(n)
    
    def MakeDoor(self, r1: Room, r2: Room):
        return Door(r1, r2)

Note that `MazeFactory` is not an abstract class; thus it acts as both the `AbstractFactory` and the `ConcreteFactory`. This is another common implementation for simple applications of the `Abstract Factory` pattern. Because the `MazeFactory` is a concrete class consisting entirely of factory methods, it's easy to make a new `MazeFactory` by making a subclass and overriding the operations that need to change.

In [ ]:
class MazeGame:
    @staticmethod
    def CreateMaze(factory: MazeFactory) -> Maze:
        aMaze = factory.MakeMaze()
        r1 = factory.MakeRoom(1)
        r2 = factory.MakeRoom(2)
        aDoor = factory.MakeDoor(r1, r2)

        aMaze.AddRoom(r1)
        aMaze.AddRoom(r2)

        r1.SetSide(Direction.NORTH, factory.MakeWall())
        r1.SetSide(Direction.EAST, aDoor)
        r1.SetSide(Direction.SOUTH, factory.MakeWall())
        r1.SetSide(Direction.WEST, factory.MakeWall())
        
        r2.SetSide(Direction.NORTH, factory.MakeWall())
        r2.SetSide(Direction.EAST, factory.MakeWall())
        r2.SetSide(Direction.SOUTH, factory.MakeWall())
        r2.SetSide(Direction.WEST, aDoor)
        
        return aMaze

For example, it is possible to create `EnchantedMazeFactory`, that is a factory for enchanted mazes, by subclassing MazeFactory.

In [ ]:
class EnchantedRoom(Room):
    def __init__(self, roomNo, CastSpell):
        super().__init__(roomNo)

class DoorNeedingSpell(Door):
    def __init__(self, r1, r2):
        super().__init__(r1, r2)

class EnchantedMazeFactory(MazeFactory):
    def MakeRoom(self, n):
        return EnchantedRoom(n, self.CastSpell())
    
    def MakeDoor(self, r1:Room, r2:Room):
        return DoorNeedingSpell(r1, r2)
    
    def CastSpell(self):
        print("Entered Enchanted Room")

Now suppose we want to make a maze game in which a room can have a bomb set in it. If the bomb goes off, it will damage the walls (at least). We can make a subclass of Room keep track of whether the room has a bomb in it and whether the bomb has gone off. We'll also need a subclass of Wall to keep track of the damage done to the wall. We'll call these classes RoomWithABomb and BombedWall.

The last class we'll define is BombedMazeFactory, a subclass of MazeFactory that ensures walls are of class BombedWall and rooms are of class RoomWithABomb. BombedMazeFactory only needs to override two functions:

```cpp
Wall* BombedMazeFactory::MakeWall () const {
return new BombedWall;
}
Room* BombedMazeFactory::MakeRoom(int n) const {
return new RoomWithABomb(n);
}
```

To build a simple maze that can contain bombs, we simply call CreateMaze with a BombedMazeFactory.

```cpp
MazeGame game;
BombedMazeFactory factory;
game.CreateMaze(factory);
```

## Builder

`MazeBuilder` does not create mazes itself; its main purpose is just to define an interface for creating mazes. It defines empty implementations primarily for convenience. Subclasses of `MazeBuilder` do the actual work.

In [ ]:
class MazeBuilder(ABC):
    def BuildMaze():
        pass
    def BuildRoom(room:int):
        pass
    def BuildDoor(roomFrom:int, roomTo: int):
        pass
    @abstractmethod
    def GetMaze():
        pass

class MazeGame:
    @staticmethod
    def CreateMaze(builder: MazeBuilder):
        builder.BuildMaze()
        builder.BuildRoom(1)
        builder.BuildRoom(2)
        builder.BuildDoor(1, 2)

        return builder.GetMaze()


Any other `MazeGame` can be implemented to build a maze with the required structure.

In [ ]:
class StandardMazeBuilder(MazeBuilder):
    def __init__(self):
        _currentMaze = None # keep track of the maze it's built
    def BuildMaze(self):
        _currentMaze = Maze()
    def BuildRoom(self, n:int):
        if n not in self._currentMaze:
            room = Room(n)
            self._currentMaze.AddRoom(room)
            room.SetSide(Direction.NORTH, Wall())
            room.SetSide(Direction.EAST, Wall())
            room.SetSide(Direction.SOUTH, Wall())
            room.SetSide(Direction.WEST, Wall())
        pass
    def BuildDoor(self, n1:int, n2: int):
        r1 = self._currentMaze.RoomNo(n1)
        r2 = self._currentMaze.RoomNo(n2)
        d = Door(r1, r2)
        r1.SetSide(self.CommonWall(r1, r2), d)
        r2.SetSide(self.CommonWall(r2, r1), d)
    def GetMaze(self):
        return self._current_maze
    def CommonWall(r1:Room, r2:Room):
        "utylity operation that determines the direction of the common wall between two rooms"
        pass

In [ ]:
maze = Maze()
game = MazeGame()
builder = StandardMazeBuilder()
game.CreateMaze(builder)
maze = builder.GetMaze()

`CountingMazeBuilder` jsut counts the different kinds of components that would have been created.

In [ ]:
class CountingMazeBuilder(MazeBuilder):
    def __init__(self):
        self._doors = 0
        self._rooms = 0
    def BuildMaze():
        pass
    def BuildRoom(self, room:int):
        self._rooms += 1
    def BuildDoor(self, roomFrom:int, roomTo: int):
        self._doors += 1
    def AddWall(n:int, direction:Direction):
        pass
    def GetCounts(n:int, m:int):
        rooms = self._rooms
        doors = self._doors
        return rooms, doors
    def GetMaze():
        pass


game = MazeGame()
builder = CountingMazeBuilder()
game.CreateMaze(builder)
rooms, doors = builder.GetCounts()
print(f"The maze has {rooms} rooms and {doors} doors.")    

## Factory Method

The `Factory Method` to let subclasses choose the components (i.e. maze, rooms, doors, and walls).

Each factory method returns a maze component of a given type. `MazeGame` provides default implementations that return the simplest kinds of maze, rooms, walls, and doors.

In [ ]:
class MazeGame:
    def CreateMaze(self):
        aMaze = self.MakeMaze()
        r1 = self.MakeRoom(1)
        r2 = self.MakeRoom(2)
        theDoor = self.MakeDoor(r1, r2)
        r1.SetSide(Direction.NORTH, self.MakeWall())
        r1.SetSide(Direction.EAST, theDoor)
        r1.SetSide(Direction.SOUTH, self.MakeWall())
        r1.SetSide(Direction.WEST, self.MakeWall())
        r2.SetSide(Direction.NORTH, self.MakeWall())
        r2.SetSide(Direction.EAST, self.MakeWall)
        r2.SetSide(Direction.SOUTH, self.MakeWall())
        r2.SetSide(Direction.WEST, theDoor)
        return aMaze
    def MakeMaze(self):
        pass
    def MakeRoom(self, n: int):
        return Room(n)
    def MakeWall(self):
        return Wall()
    def MakeDoor(self, r1:Room, r2:Room):
        return Door(r1, r2)

Different games can subclass `MazeGame` to specialize parts of the maze. `MazeGame` subclasses can redefine some or all of the factory methods to specify variations in products. For example, a `BombedMazeGame` can redefine the Room and Wall products to return the bombed varieties:

In [ ]:
class BombedMazeGame(MazeGame):
    def __init__(self):
        pass
    def MakeWall(self):
        return BombedWall()
    def MakeRoom(n:int):
        return RoomWithABomb(n)

An `EnchantedMazeGame` variant might be defined like this:

In [ ]:
from . import Room

class EnchantedMazeGame(MazeGame):
    def __init__(self):
        pass
    def MakeRoom(self, n: int):
        return EnchantedRoom(n, self.CastSpell())
    def MakeDoor(self, r1: Room, r2: Room):
        return DoorNeedingSpell(r1, r2)
    def CastSpell():
        pass

## Prototype

`MazePrototypeFactory` will be initialized with prototypes of the objects it will create so that we don't have to subclass it just to change the classes of walls or rooms it creates.

In [ ]:
class MazePrototypeFactory(MazeFactory):
    def __init__(self, maze:Maze, wall:Wall, room:Room, door:Door):
        self._prototypeMaze = maze
        self._prototypeWall = wall
        self._prototypeRoom = room
        self._prototypeDoor = door

    def MakeWall(self):
        return self._prototypeWall.clone()
    
    def makeDoor(self, r1:Room, r2:Room):
        door = self._prototypeDoor.clone()
        door.initialize(r1, r2)
        return door

Usage:

In [ ]:
game = MazeGame()
simpleMazeFactory = MazePrototypeFactory(Maze(), Wall(), Room(), Door())
maze = game.CreateMaze(simpleMazeFactory)

An object to be used as a prototype would need having:

- `.clone()` operation
- copy constructor for cloning
- `Initialize` operation for reinitializing the internal state

In [ ]:
from copy import deepcopy

class Door(MapSite):
    def __init__(self, other:Door):
        self._room1 = other._room1
        self._room2 = other._room2

    def Initialize(self, r1:Room, r2:Room):
        _room1 = r1
        _room2 = r2

    def Clone(self):
        return deepcopy(self)

Another maze type:

In [ ]:
class BombedWall(Wall):
    def __init__(self, other):
        self._bomb = other._bomb

    def Clone(self):
        return deepcopy(self)

bombedMazeFactory = MazePrototypeFactory(Maze(), BombedWall(), RommWithABomb(), Door())

## Singleton

By making the `MazeFactory` a singleton, we make the maze object globally accessible without resorting to global variables.

We mae it a singleton class by adding a static `Instance` operation and a static `_instance` member to hold the one and only one instance.

In [22]:
class MazeFactorySingleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(MazeFactorySingleton, cls).__new__(cls)
        return cls._instance
    
    def MakeMaze():
        return Maze()
    
    def MakeWall():
        return Wall()
    
    def MakeRoom(n: int):
        return Room(n)
    
    def MakeDoor(r1: Room, r2: Room):
        return Door(r1, r2)
    
    def Instance(self):
        if self._instance == 0:
            self._instance = 1
        return self._instance

In [26]:
mfs = MazeFactorySingleton()
mfs2 = MazeFactorySingleton()
mfs is mfs2

True